In [1]:
import boxball_loader as bbl

In [3]:
glt = bbl.load_gamelog_teams(game_types=bbl.GameType.ALL, years=range(1800, 3000))

In [5]:
glt.columns

Index(['game_id', 'date', 'double_header', 'yr', 'game_type', 'park_id',
       'team', 'team_league', 'team_game_number', 'runs_scored', 'line_score',
       'ab', 'h', 'd', 't', 'hr', 'rbi', 'sh', 'sf', 'hbp', 'bb', 'ibb', 'k',
       'sb', 'cs', 'gdp', 'ci', 'lob', 'pitchers', 'er', 'ter', 'wp', 'balks',
       'po', 'a', 'e', 'passed', 'db', 'tp', 'manager_id', 'manager_name',
       'starting_pitcher_id', 'starting_pitcher_name', 'batting_1_player_id',
       'batting_1_name', 'batting_1_position', 'batting_2_player_id',
       'batting_2_name', 'batting_2_position', 'batting_3_player_id',
       'batting_3_name', 'batting_3_position', 'batting_4_player_id',
       'batting_4_name', 'batting_4_position', 'batting_5_player_id',
       'batting_5_name', 'batting_5_position', 'batting_6_player_id',
       'batting_6_name', 'batting_6_position', 'batting_7_player_id',
       'batting_7_name', 'batting_7_position', 'batting_8_player_id',
       'batting_8_name', 'batting_8_position', '

In [24]:
glt[glt['opp']=='SDN'].sort_values(['date', 'double_header']).rolling(3).agg({'k': sum}).max()

k    42.0
dtype: float64

In [82]:
sdn = glt[glt['opp']=='SDN'].sort_values(['date', 'double_header'])
for i in range(2, 30):
    sdn = pd.merge(sdn, sdn.rolling(i)['k'].sum().rename(f'k_rolling{i}'), left_index=True, right_index=True)
    print (sdn.sort_values(f'k_rolling{i}', ascending=False).head(1)[['date', f'k_rolling{i}']])

            date  k_rolling2
41782 2001-06-19        33.0
           date  k_rolling3
8799 2019-09-19        42.0
           date  k_rolling4
8812 2019-09-20        55.0
           date  k_rolling5
8812 2019-09-20        65.0
           date  k_rolling6
8827 2019-09-21        74.0
           date  k_rolling7
8842 2019-09-22        84.0
           date  k_rolling8
8876 2019-09-25        95.0
           date  k_rolling9
8890 2019-09-26       106.0
           date  k_rolling10
8890 2019-09-26        116.0
           date  k_rolling11
8890 2019-09-26        125.0
           date  k_rolling12
8812 2019-09-20        135.0
           date  k_rolling13
8812 2019-09-20        146.0
           date  k_rolling14
8812 2019-09-20        157.0
           date  k_rolling15
8827 2019-09-21        166.0
             date  k_rolling16
161494 2020-09-23        176.0
           date  k_rolling17
8876 2019-09-25        186.0
           date  k_rolling18
8876 2019-09-25        197.0
           date  k_rolli

In [26]:
dailies = bbl.load_dailies_pit(game_types=bbl.GameType.ALL)
dailies.columns

Index(['game_id', 'game_dt', 'game_ct', 'appearance_dt', 'team_id',
       'player_id', 'slot_ct', 'seq_ct', 'home_fl', 'opponent_id', 'park_id',
       'yr', 'game_type', 'team_game_number', 'p_g', 'p_gs', 'p_cg', 'p_sho',
       'p_gf', 'p_w', 'p_l', 'p_sv', 'p_out', 'p_tbf', 'p_ab', 'p_r', 'p_er',
       'p_h', 'p_tb', 'p_2b', 'p_3b', 'p_hr', 'p_hr4', 'p_bb', 'p_ibb', 'p_so',
       'p_gdp', 'p_hp', 'p_sh', 'p_sf', 'p_xi', 'p_wp', 'p_bk', 'p_ir',
       'p_irs', 'p_go', 'p_ao', 'p_pitch', 'p_strike'],
      dtype='object')

In [86]:
sdn = glt[glt['opp']=='SDN'].sort_values(['date', 'double_header'])

pd.concat([pd.merge(sdn, sdn.rolling(i)['k'].sum().rename('sum_k'), left_index=True, right_index=True).sort_values('sum_k', ascending=False).head(1)[['date', 'sum_k']] for i in range(2, 30)])

,date,sum_k
41782,2001-06-19,33.0
8799,2019-09-19,42.0
8812,2019-09-20,55.0
8812,2019-09-20,65.0
8827,2019-09-21,74.0
8842,2019-09-22,84.0
8876,2019-09-25,95.0
8890,2019-09-26,106.0
8890,2019-09-26,116.0
8890,2019-09-26,125.0


In [30]:
sdn = dailies[dailies['team_id']=='SDN'].sort_values(['game_dt', 'game_ct'])
sdn

,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,p_sf,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike
4095591,SDN196904080,1969-04-08,0,1969-04-08,SDN,selmd101,9,1,True,HOU,...,0.0,0.0,0.0,0,0.0,0.0,6.0,9.0,NaN,NaN
4095612,SDN196904090,1969-04-09,0,1969-04-09,SDN,podrj101,9,1,True,HOU,...,0.0,0.0,0.0,0,0.0,0.0,9.0,11.0,NaN,NaN
4095614,SDN196904090,1969-04-09,0,1969-04-09,SDN,siskt101,9,3,True,HOU,...,0.0,0.0,0.0,0,0.0,0.0,3.0,3.0,NaN,NaN
4095635,SDN196904100,1969-04-10,0,1969-04-10,SDN,kelld102,9,1,True,HOU,...,0.0,0.0,0.0,0,0.0,0.0,6.0,11.0,NaN,NaN
4095636,SDN196904100,1969-04-10,0,1969-04-10,SDN,mccob105,9,3,True,HOU,...,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4209845,SDN202010080,2020-10-08,0,2020-10-08,SDN,pomed001,0,9,True,LAN,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,22.0,11.0
4209847,SDN202010080,2020-10-08,0,2020-10-08,SDN,richg002,0,10,True,LAN,...,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,10.0,7.0
4209848,SDN202010080,2020-10-08,0,2020-10-08,SDN,roset001,0,11,True,LAN,...,0.0,0.0,0.0,0,0.0,0.0,0.0,2.0,29.0,13.0
4209849,SDN202010080,2020-10-08,0,2020-10-08,SDN,stamc001,0,2,True,LAN,...,0.0,0.0,0.0,0,1.0,1.0,1.0,0.0,20.0,11.0


In [33]:
sdn.groupby('game_id')['p_so'].sum().rolling(3).sum().max()

42.0

In [42]:
sdn['game_dt'].min()

Timestamp('1969-04-08 00:00:00')

In [49]:
def game_score_raw(outs, bb, so, h, hr, r):
    return 40 + 2*outs + so - 2*bb - 2*h - 3*r - 6*hr

In [50]:
def game_score(row):
    return game_score_raw(row['p_out'], row['p_bb'], row['p_so'], row['p_h'], row['p_hr'], row['p_r'])

In [51]:
game_score()

4095591    89.0
4095612    73.0
4095635    87.0
4095659    34.0
4095673    41.0
           ... 
4209760    25.0
4209812    48.0
2582607    41.0
2582641    37.0
4209838    37.0
Length: 8238, dtype: float64

In [52]:
starts = sdn[sdn['p_gs']>0]
starts['p_gsc'] = game_score(starts)
starts

<ipython-input-52-3e4c81728ca8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts['p_gsc'] = game_score(starts)


,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike,p_gsc
4095591,SDN196904080,1969-04-08,0,1969-04-08,SDN,selmd101,9,1,True,HOU,...,0.0,0.0,0,0.0,0.0,6.0,9.0,NaN,NaN,89.0
4095612,SDN196904090,1969-04-09,0,1969-04-09,SDN,podrj101,9,1,True,HOU,...,0.0,0.0,0,0.0,0.0,9.0,11.0,NaN,NaN,73.0
4095635,SDN196904100,1969-04-10,0,1969-04-10,SDN,kelld102,9,1,True,HOU,...,0.0,0.0,0,0.0,0.0,6.0,11.0,NaN,NaN,87.0
4095659,SDN196904110,1969-04-11,0,1969-04-11,SDN,kirbc101,9,1,True,SFN,...,0.0,1.0,0,0.0,0.0,2.0,7.0,NaN,NaN,34.0
4095673,SDN196904120,1969-04-12,0,1969-04-12,SDN,mcbea101,9,1,True,SFN,...,0.0,1.0,0,0.0,0.0,7.0,10.0,NaN,NaN,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4209760,SDN202010010,2020-10-01,0,2020-10-01,SDN,daviz001,0,1,True,SLN,...,0.0,0.0,0,0.0,0.0,3.0,0.0,55.0,35.0,25.0
4209812,SDN202010020,2020-10-02,0,2020-10-02,SDN,stamc001,0,1,True,SLN,...,0.0,0.0,0,0.0,0.0,2.0,3.0,21.0,14.0,48.0
2582607,LAN202010060,2020-10-06,0,2020-10-06,SDN,clevm001,0,1,False,LAN,...,0.0,1.0,0,0.0,0.0,0.0,2.0,24.0,10.0,41.0
2582641,LAN202010070,2020-10-07,0,2020-10-07,SDN,daviz001,0,1,False,LAN,...,0.0,0.0,0,0.0,0.0,7.0,4.0,73.0,49.0,37.0


In [53]:
starts['p_gsc'].max()

114.0

In [54]:
starts.sort_values('p_gsc')

,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,opponent_id,...,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike,p_gsc
1835505,COL199907031,1999-07-03,1,1999-07-03,SDN,willw001,9,1,False,COL,...,0.0,1.0,0,0.0,0.0,1.0,4.0,97.0,58.0,-23.0
4732828,SLN200108300,2001-08-30,0,2001-08-30,SDN,hernj003,9,1,False,SLN,...,0.0,1.0,0,0.0,0.0,3.0,5.0,89.0,46.0,-18.0
1566921,CIN201108130,2011-08-13,0,2011-08-13,SDN,staut001,9,1,False,CIN,...,0.0,0.0,0,0.0,0.0,5.0,3.0,79.0,46.0,-16.0
1545400,CIN200208090,2002-08-09,0,2002-08-09,SDN,tomkb001,9,1,False,CIN,...,0.0,0.0,0,0.0,0.0,4.0,3.0,74.0,42.0,-15.0
2572499,LAN201607080,2016-07-08,0,2016-07-08,SDN,casha001,9,1,False,LAN,...,0.0,0.0,0,0.0,0.0,1.0,4.0,69.0,34.0,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4158632,SDN199808161,1998-08-16,1,1998-08-16,SDN,browk001,9,1,True,MIL,...,0.0,0.0,0,0.0,0.0,12.0,4.0,126.0,90.0,99.0
4149892,SDN199407030,1994-07-03,0,1994-07-03,SDN,benea001,9,1,True,NYN,...,0.0,1.0,0,0.0,0.0,7.0,8.0,116.0,82.0,103.0
4107353,SDN197408042,1974-08-04,2,1974-08-04,SDN,freid101,9,1,True,CIN,...,0.0,0.0,0,0.0,0.0,9.0,21.0,NaN,NaN,103.0
4102529,SDN197206072,1972-06-07,2,1972-06-07,SDN,kirbc101,9,1,True,PIT,...,0.0,0.0,0,0.0,0.0,14.0,15.0,NaN,NaN,104.0


In [56]:
starts.rolling(3)['p_gsc'].sum().max()

253.0

In [62]:
import pandas as pd
starts = pd.merge(starts.rolling(3)['p_gsc'].sum().rename('gsc_rolling3'), starts, left_index=True, right_index=True)
starts.sort_values(['gsc_rolling3'])

,gsc_rolling3,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,home_fl,...,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike,p_gsc
4045653,18.0,PIT200007300,2000-07-30,0,2000-07-30,SDN,meadb001,9,1,False,...,0.0,0.0,0,0.0,0.0,2.0,13.0,102.0,59.0,9.0
1835528,26.0,COL199907032,1999-07-03,2,1999-07-03,SDN,murrh001,9,1,False,...,0.0,0.0,0,0.0,0.0,5.0,5.0,70.0,37.0,0.0
4166013,27.0,SDN200108311,2001-08-31,1,2001-08-31,SDN,jodib001,9,1,True,...,0.0,0.0,0,0.0,0.0,7.0,4.0,67.0,39.0,50.0
1879106,30.0,COL201707190,2017-07-19,0,2017-07-19,SDN,richc002,9,1,False,...,0.0,0.0,0,0.0,0.0,6.0,2.0,100.0,63.0,-10.0
30235,32.0,ANA200906140,2009-06-14,0,2009-06-14,SDN,younc003,0,1,False,...,0.0,0.0,0,0.0,0.0,0.0,6.0,69.0,35.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123195,252.0,SDN198206120,1982-06-12,0,1982-06-12,SDN,welsc001,9,1,True,...,0.0,0.0,0,0.0,0.0,13.0,10.0,NaN,NaN,86.0
4108158,253.0,SDN197504230,1975-04-23,0,1975-04-23,SDN,mcinj101,9,1,True,...,0.0,0.0,0,0.0,0.0,11.0,10.0,NaN,NaN,80.0
3305068,253.0,NYN197205050,1972-05-05,0,1972-05-05,SDN,arlis101,9,1,False,...,0.0,0.0,0,0.0,0.0,11.0,7.0,NaN,NaN,83.0
4095591,NaN,SDN196904080,1969-04-08,0,1969-04-08,SDN,selmd101,9,1,True,...,0.0,0.0,0,0.0,0.0,6.0,9.0,NaN,NaN,89.0


In [89]:

starts = pd.merge(starts.rolling(3)['p_so'].sum().rename('so_rolling3'), starts, left_index=True, right_index=True)
starts.sort_values(['so_rolling3'])

,so_rolling3,gsc_rolling3,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,seq_ct,...,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike,p_gsc
4114205,0.0,83.0,SDN197709170,1977-09-17,0,1977-09-17,SDN,freid101,9,1,...,0.0,0.0,0,0.0,0.0,2.0,3.0,NaN,NaN,12.0
4668802,0.0,54.0,SLN197208270,1972-08-27,0,1972-08-27,SDN,kirbc101,9,1,...,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,NaN,23.0
4110394,0.0,84.0,SDN197604290,1976-04-29,0,1976-04-29,SDN,strob101,9,1,...,0.0,0.0,0,0.0,0.0,2.0,4.0,NaN,NaN,29.0
4121678,0.0,119.0,SDN198108160,1981-08-16,0,1981-08-16,SDN,muras001,9,1,...,0.0,0.0,0,0.0,0.0,6.0,14.0,NaN,NaN,66.0
4135362,1.0,111.0,SDN198709300,1987-09-30,0,1987-09-30,SDN,hawka001,9,1,...,0.0,0.0,0,0.0,0.0,7.0,3.0,NaN,NaN,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208630,30.0,176.0,SDN201909200,2019-09-20,0,2019-09-20,SDN,lauee001,9,1,...,0.0,0.0,0,0.0,0.0,1.0,8.0,106.0,70.0,67.0
4158788,30.0,209.0,SDN199808310,1998-08-31,0,1998-08-31,SDN,hamij002,9,1,...,0.0,0.0,0,0.0,0.0,7.0,5.0,107.0,67.0,59.0
4197314,31.0,169.0,SDN201504290,2015-04-29,0,2015-04-29,SDN,casha001,9,1,...,0.0,0.0,0,0.0,0.0,3.0,8.0,110.0,75.0,63.0
4095591,NaN,NaN,SDN196904080,1969-04-08,0,1969-04-08,SDN,selmd101,9,1,...,0.0,0.0,0,0.0,0.0,6.0,9.0,NaN,NaN,89.0


In [90]:
starts = pd.merge(starts.rolling(4)['p_so'].sum().rename('so_rolling4'), starts, left_index=True, right_index=True)
starts.sort_values(['so_rolling4'])

,so_rolling4,so_rolling3,gsc_rolling3,game_id,game_dt,game_ct,appearance_dt,team_id,player_id,slot_ct,...,p_xi,p_wp,p_bk,p_ir,p_irs,p_go,p_ao,p_pitch,p_strike,p_gsc
4687374,2.0,2.0,163.0,SLN198108170,1981-08-17,0,1981-08-17,SDN,wiser101,9,...,0.0,0.0,0,0.0,0.0,10.0,7.0,NaN,NaN,67.0
4096779,2.0,2.0,94.0,SDN196907100,1969-07-10,0,1969-07-10,SDN,robed101,9,...,0.0,0.0,0,0.0,0.0,0.0,0.0,NaN,NaN,26.0
2039665,2.0,2.0,57.0,DET198410140,1984-10-14,0,1984-10-14,SDN,thurm001,0,...,0.0,0.0,0,0.0,0.0,1.0,0.0,15.0,9.0,17.0
4110394,2.0,0.0,84.0,SDN197604290,1976-04-29,0,1976-04-29,SDN,strob101,9,...,0.0,0.0,0,0.0,0.0,2.0,4.0,NaN,NaN,29.0
2039599,2.0,2.0,78.0,DET198410120,1984-10-12,0,1984-10-12,SDN,lollt001,0,...,0.0,1.0,0,0.0,0.0,2.0,3.0,55.0,28.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197329,37.0,25.0,162.0,SDN201505010,2015-05-01,0,2015-05-01,SDN,kenni001,9,...,0.0,0.0,0,0.0,0.0,6.0,5.0,101.0,68.0,52.0
4208630,39.0,30.0,176.0,SDN201909200,2019-09-20,0,2019-09-20,SDN,lauee001,9,...,0.0,0.0,0,0.0,0.0,1.0,8.0,106.0,70.0,67.0
4095591,NaN,NaN,NaN,SDN196904080,1969-04-08,0,1969-04-08,SDN,selmd101,9,...,0.0,0.0,0,0.0,0.0,6.0,9.0,NaN,NaN,89.0
4095612,NaN,NaN,NaN,SDN196904090,1969-04-09,0,1969-04-09,SDN,podrj101,9,...,0.0,0.0,0,0.0,0.0,9.0,11.0,NaN,NaN,73.0
